# debug datarmor

tests are conducted with zarr first and then with netcdf on the rotary spectrum computation

In [1]:
import distributed, dask
print(distributed.__version__)
print(dask.__version__)
xr.show_versions()

/home1/datahome/aponte/.miniconda3/envs/equinox/lib/python3.6/site-packages/distributed/utils.py:134: RuntimeWarning: Couldn't detect a suitable IP address for reaching '8.8.8.8', defaulting to '127.0.0.1': [Errno 101] Network is unreachable
  % (host, default, e), RuntimeWarning)


1.23.2
0.19.2


NameError: name 'xr' is not defined

In [1]:
import os, sys
import numpy as np
import dask
import xarray as xr
from matplotlib import pyplot as plt
%matplotlib inline

import datetime
now = datetime.datetime.now

from mitequinox.utils import *

/home1/datahome/aponte/.miniconda3/envs/equinox/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home1/datahome/aponte/.miniconda3/envs/equinox/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home1/datahome/aponte/.miniconda3/envs/equinox/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [20]:
from dask_jobqueue import PBSCluster
#cluster = PBSCluster()
# default:    cores: 8   processes: 2    memory: 100GB
#w = cluster.scale(32)
#cluster = PBSCluster(cores=2, processes=2, memory='100G', resource_spec='select=1:ncpus=28:mem=110GB') # conservative memory usage: ~40-70GB total for standard zarr
#cluster = PBSCluster(cores=4, processes=2, memory='100G', resource_spec='select=1:ncpus=28:mem=110GB') # faster, ~50-80GB total
cluster = PBSCluster(cores=4, processes=2, memory='50G', resource_spec='select=1:ncpus=28:mem=110GB') #
w = cluster.scale(20)

In [21]:
# get dask handles and check dask server status
from dask.distributed import Client
client = Client(cluster)

In [22]:
client

Client Scheduler: tcp://10.148.0.39:33787 Dashboard: http://10.148.0.39:8787/status,Cluster Workers: 20 Cores: 40 Memory: 500.00 GB


In [5]:
# main output directory
scratchd = '/home/c11-data/Test_aponte/'

In [6]:
# zarr compression
# http://xarray.pydata.org/en/stable/io.html
# http://zarr.readthedocs.io/en/stable/tutorial.html#compressors
compressor = None

---

# store grid

In [7]:
ds_index, ds = get_compressed_level_index(grid_dir)

# we will need to keep some of these when computing gradients
#ds = ds.drop(['dxG','dyG','dxC','dyC','rAw','rAs','rAz'])
ds = ds.drop(['hFacC','hFacW','hFacS'])
ds = ds.drop(['Z', 'Zp1', 'Zu', 'Zl', 'drC', 'drF','PHrefC','PHrefF'])

Nc = 432 # original choice
#Nc = 27 # very long scheduling
#Nc = 96 # 96*45
ds = ds.chunk({'i': Nc, 'j': Nc, 'i_g': Nc, 'j_g': Nc})

#print(ds)

/home1/datahome/aponte/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/mds_store.py:735: UserWarning: Couldn't find available_diagnostics.log in . Using default version.
  "in %s. Using default version." % data_dir)
/home1/datahome/aponte/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/utils.py:336: UserWarning: Not sure what to do with rlev = L
  warnings.warn("Not sure what to do with rlev = " + rlev)
/home1/datahome/aponte/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/mds_store.py:249: FutureWarning: iteration over an xarray.Dataset will change in xarray v0.11 to only include data variables, not coordinates. Iterate over the Dataset.variables property instead to preserve existing behavior in a forwards compatible manner.
  for vname in ds:


In [8]:
out_dir = scratchd+'mit/'

In [9]:
file_out = out_dir+'grid.zarr'
print(now())
%time ds.to_zarr(file_out, mode='w')

2018-10-17 10:30:15.418837
CPU times: user 25.8 s, sys: 1.13 s, total: 26.9 s
Wall time: 37.3 s


--- 

# zarr case

In [23]:
V = ['SSU', 'SSV']

# faces
F = range(13)
F = range(2)

# parameters for rechunking
Nt = 24*10 # time chunks
#
#Nc = 96 # x 45 = 4320
#Nc = 48 # x 90 = 4320
#Nc = (96, 48)
# other choices: 432, 27 (long scheduling), 288, 864
Nc = (120, 60) # for dij slices

In [ ]:
print('++ zarr conversion, standard layout: ', now())
%time zarr_standard(['SSU'], client, out_dir=scratchd+'mit/standard/')

++ zarr conversion, standard layout:  2018-10-18 18:42:01.556029


/home1/datahome/aponte/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/mds_store.py:735: UserWarning: Couldn't find available_diagnostics.log in . Using default version.
  "in %s. Using default version." % data_dir)
/home1/datahome/aponte/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/utils.py:336: UserWarning: Not sure what to do with rlev = L
  warnings.warn("Not sure what to do with rlev = " + rlev)
/home1/datahome/aponte/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/mds_store.py:249: FutureWarning: iteration over an xarray.Dataset will change in xarray v0.11 to only include data variables, not coordinates. Iterate over the Dataset.variables property instead to preserve existing behavior in a forwards compatible manner.
  for vname in ds:


In [ ]:
face=0
print('++ zarr conversion, standard layout: ', now())
%time zarr_standard(['SSU'], client,  F=face, out_dir=scratchd+'mit/standard/')
print('++ zarr rechunking and storage: ', now())
%time rechunk(['SSU'], F=face, in_dir=scratchd+'mit/standard/', out_dir=scratchd+'mit/rechunked/', \
              Nt=Nt, Nc=Nc, dij=4)

++ zarr conversion, standard layout:  2018-10-18 15:39:47.469795


/home1/datahome/aponte/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/mds_store.py:735: UserWarning: Couldn't find available_diagnostics.log in . Using default version.
  "in %s. Using default version." % data_dir)
/home1/datahome/aponte/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/utils.py:336: UserWarning: Not sure what to do with rlev = L
  warnings.warn("Not sure what to do with rlev = " + rlev)
/home1/datahome/aponte/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/mds_store.py:249: FutureWarning: iteration over an xarray.Dataset will change in xarray v0.11 to only include data variables, not coordinates. Iterate over the Dataset.variables property instead to preserve existing behavior in a forwards compatible manner.
  for vname in ds:


In [ ]:
face=1
print('++ zarr conversion, standard layout: ', now())
%time zarr_standard(['SSU'], client,  F=face, out_dir=scratchd+'mit/standard/')
print('++ zarr rechunking and storage: ', now())
%time rechunk(['SSU'], F=face, in_dir=scratchd+'mit/standard/', out_dir=scratchd+'mit/rechunked/', \
              Nt=Nt, Nc=Nc, dij=4)

In [12]:
# try to reintore compression once the computation goes through
# try to delete del statements

#overwrite = False
overwrite = True

for face in F:

    Efile = scratchd+'mit/spectra/%s_f%02d.zarr'%('SSUV',face)
    
    if not os.path.isdir(Efile) or overwrite:
    
        if True:
            # binary to zarr (standard layout)
            print('++ zarr conversion, standard layout: ', now())
            %time zarr_standard(['SSU'], client,  F=face, out_dir=scratchd+'mit/standard/')
            #%time zarr_standard(['SSV'], client,  F=face, out_dir=scratchd+'mit/standard/')

        # rechunk all variables
        print('++ zarr rechunking and storage: ', now())
        %time rechunk(['SSU'], F=face, in_dir=scratchd+'mit/standard/', out_dir=scratchd+'mit/rechunked/', \
                      Nt=Nt, Nc=Nc, dij=4)
        #%time rechunk(['SSV'], F=face, in_dir=scratchd+'mit/standard/', out_dir=scratchd+'mit/rechunked/', \
        #              Nt=Nt, Nc=Nc, dij=4)
    
    print('--- face %d done'%face)

++ zarr conversion, standard layout:  2018-10-18 14:33:42.874646


/home1/datahome/aponte/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/mds_store.py:735: UserWarning: Couldn't find available_diagnostics.log in . Using default version.
  "in %s. Using default version." % data_dir)
/home1/datahome/aponte/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/utils.py:336: UserWarning: Not sure what to do with rlev = L
  warnings.warn("Not sure what to do with rlev = " + rlev)
/home1/datahome/aponte/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/mds_store.py:249: FutureWarning: iteration over an xarray.Dataset will change in xarray v0.11 to only include data variables, not coordinates. Iterate over the Dataset.variables property instead to preserve existing behavior in a forwards compatible manner.
  for vname in ds:


 SSU converted to zarr,  data is 818.4GB 
CPU times: user 5min 53s, sys: 21.1 s, total: 6min 14s
Wall time: 17min 14s
++ zarr rechunking and storage:  2018-10-18 14:50:57.130756
 SSU face=0  data is 48.0GB 
CPU times: user 8min 10s, sys: 29.1 s, total: 8min 39s
Wall time: 26min 1s
--- face 0 done
++ zarr conversion, standard layout:  2018-10-18 15:16:58.300667


/home1/datahome/aponte/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/mds_store.py:735: UserWarning: Couldn't find available_diagnostics.log in . Using default version.
  "in %s. Using default version." % data_dir)
/home1/datahome/aponte/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/utils.py:336: UserWarning: Not sure what to do with rlev = L
  warnings.warn("Not sure what to do with rlev = " + rlev)
/home1/datahome/aponte/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/mds_store.py:249: FutureWarning: iteration over an xarray.Dataset will change in xarray v0.11 to only include data variables, not coordinates. Iterate over the Dataset.variables property instead to preserve existing behavior in a forwards compatible manner.
  for vname in ds:


KilledWorker: ("('concatenate-from-value-load_level_from_3D_field-getitem-2beaaa7b052c20209ec8e55cdd5fc4e7', 5672, 0, 0, 0)", 'tcp://10.148.1.105:34989')

++ zarr rechunking and storage:  2018-10-18 15:19:56.883568


KeyboardInterrupt: 

--- face 1 done


In [8]:
# try to reintore compression once the computation goes through
# try to delete del statements

#overwrite = False
overwrite = True

for face in F:

    Efile = scratchd+'mit/spectra/%s_f%02d.zarr'%('SSUV',face)
    
    if not os.path.isdir(Efile) or overwrite:
    
        if True:
            # binary to zarr (standard layout)
            print('++ zarr conversion, standard layout: ', now())
            %time zarr_standard(V, client,  F=face, out_dir=scratchd+'mit/standard/')

        # rechunk all variables
        print('++ zarr rechunking and storage: ', now())
        %time rechunk(V, F=face, in_dir=scratchd+'mit/standard/', out_dir=scratchd+'mit/rechunked/', \
                      Nt=Nt, Nc=Nc, dij=4)

        # load data
        dsu = xr.open_zarr(scratchd+'mit/rechunked/%s_f%02d.zarr'%('SSU',face))
        dsv = xr.open_zarr(scratchd+'mit/rechunked/%s_f%02d.zarr'%('SSV',face))
        ds = xr.merge([dsu,dsv], compat='identical')

        # try to delete this file writing once the computation goes through
        #ds['SSU'] = ds['SSU'].where(abs(ds['SSU'])<10) # tmp: SSU.0000659952.shrunk is corrupted
        ds['SSUV'] = ds['SSU'].rename({'i_g': 'i'}).fillna(0.)  \
                     + 1j * ds['SSV'].rename({'j_g': 'j'}).fillna(0.)
        #del ds['SSU'], ds['SSV']
        #store_zarr(ds, scratch+'mit/spectra/SSUV_tmp.zarr', 
        #           encoding={'SSUV': {'compressor': None}})
        #print('SSUV stored')
        #del ds
        #ds = xr.open_zarr(scratch+'mit/spectra/SSUV_tmp.zarr')

        # need to run once to get frequency
        f, E = get_E(ds['SSUV'].isel(i=slice(2),j=slice(2)))
        E = get_E(ds['SSUV'], f=f)

        # store
        E = E.to_dataset()
        # tmp, waiting for xarray release 
        for v in E.variables:
            if hasattr(E[v],'encoding') and 'chunks' in E[v].encoding:
                del E[v].encoding['chunks']    
        #
        print('++ spectral calculation and storage: ', now())
        %time E.to_zarr(Efile, mode='w', \
                        encoding={'SSUV': {'compressor': None}})
        del E, ds

        # clean up
        #for v in V:
        #    shutil.rmtree(scratchd+'mit/standard/%s.zarr' %v)
        #    shutil.rmtree(scratchd+'mit/rechunked/%s_f%02d.zarr'%(v,face))
    
    print('--- face %d done'%face)

++ zarr conversion, standard layout:  2018-10-18 08:32:47.304933


/home1/datahome/aponte/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/mds_store.py:735: UserWarning: Couldn't find available_diagnostics.log in . Using default version.
  "in %s. Using default version." % data_dir)
/home1/datahome/aponte/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/utils.py:336: UserWarning: Not sure what to do with rlev = L
  warnings.warn("Not sure what to do with rlev = " + rlev)
/home1/datahome/aponte/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/mds_store.py:249: FutureWarning: iteration over an xarray.Dataset will change in xarray v0.11 to only include data variables, not coordinates. Iterate over the Dataset.variables property instead to preserve existing behavior in a forwards compatible manner.
  for vname in ds:


 SSU converted to zarr,  data is 820.6GB 


/home1/datahome/aponte/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/mds_store.py:735: UserWarning: Couldn't find available_diagnostics.log in . Using default version.
  "in %s. Using default version." % data_dir)
/home1/datahome/aponte/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/utils.py:336: UserWarning: Not sure what to do with rlev = L
  warnings.warn("Not sure what to do with rlev = " + rlev)
/home1/datahome/aponte/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/mds_store.py:249: FutureWarning: iteration over an xarray.Dataset will change in xarray v0.11 to only include data variables, not coordinates. Iterate over the Dataset.variables property instead to preserve existing behavior in a forwards compatible manner.
  for vname in ds:


KilledWorker: ("('concatenate-from-value-load_level_from_3D_field-getitem-6265aad203e3af58edc1efff0fe51c8d', 7274, 0, 0, 0)", 'tcp://10.148.0.206:58534')

++ zarr rechunking and storage:  2018-10-18 09:04:02.514187


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home1/datahome/aponte/.miniconda3/envs/equinox/lib/python3.6/site-packages/IPython/core/magics/execution.py", line 1229, in time
    out = eval(code, glob, local_ns)
  File "<timed eval>", line 1, in <module>
  File "/home1/datahome/aponte/mit_equinox/mitequinox/utils.py", line 413, in rechunk
    encoding={key: {'compressor': compressor} for key in ds.variables})
  File "/home1/datahome/aponte/.miniconda3/envs/equinox/lib/python3.6/site-packages/xarray/core/dataset.py", line 1291, in to_zarr
    group=group, encoding=encoding, compute=compute)
  File "/home1/datahome/aponte/.miniconda3/envs/equinox/lib/python3.6/site-packages/xarray/backends/api.py", line 859, in to_zarr
    dataset.dump_to_store(store, sync=True, encoding=encoding, compute=compute)
  File "/home1/datahome/aponte/.miniconda3/envs/equinox/lib/python3.6/site-packages/xarray/core/dataset.py", line 1181, in dump_to_store
    store.sync(compute=compute)
  File "/home1/datahome/ap


KeyboardInterrupt



KeyboardInterrupt: 

---

# netcdf

---

# zarr: rewrite full dataset with time layout

In [ ]:
V = ['SSU', 'SSV']

# faces
F = range(13)
F = range(2)

# parameters for rechunking
Nt = 24*10 # time chunks
#
#Nc = 96 # x 45 = 4320
#Nc = 48 # x 90 = 4320
#Nc = (96, 48)
# other choices: 432, 27 (long scheduling), 288, 864
Nc = (120, 60) # for dij slices

In [ ]:
# try to reintore compression once the computation goes through
# try to delete del statements

#overwrite = False
overwrite = True

for face in F:

    Efile = scratchd+'mit/spectra/%s_f%02d.zarr'%('SSUV',face)
    
    if not os.path.isdir(Efile) or overwrite:
    
        if True:
            # binary to zarr (standard layout)
            print('++ zarr conversion, standard layout: ', now())
            %time zarr_standard(V, client,  F=face, out_dir=scratchd+'mit/standard/')

        # rechunk all variables
        print('++ zarr rechunking and storage: ', now())
        %time rechunk(V, F=face, in_dir=scratchd+'mit/standard/', out_dir=scratchd+'mit/rechunked/', \
                      Nt=Nt, Nc=Nc, dij=4)
    
    print('--- face %d done'%face)

---

In [16]:
cluster.scale(32)

In [ ]:
client.restart()

In [25]:
# kill scheduler, workers
cluster.scheduler.close()
#cluster.stop_workers(cluster.jobs)

<Future pending cb=[_make_coroutine_wrapper.<locals>.wrapper.<locals>.<lambda>() at /home1/datahome/aponte/.miniconda3/envs/equinox/lib/python3.6/site-packages/tornado/gen.py:347]>

In [15]:
print(cluster.job_script())

#!/bin/bash

#!/usr/bin/env bash
#PBS -N dask-worker
#PBS -q mpi_1
#PBS -A equinox
#PBS -l select=1:ncpus=28:mem=100GB
#PBS -l walltime=24:00:00
#PBS -m n
JOB_ID=${PBS_JOBID%.*}



/home1/datahome/aponte/.miniconda3/envs/equinox/bin/python -m distributed.cli.dask_worker tcp://10.148.0.39:41451 --nthreads 4 --nprocs 2 --memory-limit 25.00GB --name dask-worker--${JOB_ID}-- --death-timeout 60 --interface  ib0 



In [2]:
from dask_jobqueue import PBSCluster

/home1/datahome/aponte/.miniconda3/envs/equinox/lib/python3.6/site-packages/distributed/utils.py:134: RuntimeWarning: Couldn't detect a suitable IP address for reaching '8.8.8.8', defaulting to '127.0.0.1': [Errno 101] Network is unreachable
  % (host, default, e), RuntimeWarning)


In [3]:
cluster = PBSCluster()
print(cluster.job_script())
cluster.scheduler.close()
cluster = PBSCluster(cores=2, processes=2, memory='100G',resource_spec='select=1:ncpus=28:mem=110GB')
print(cluster.job_script())
cluster.scheduler.close()
cluster = PBSCluster(cores=4, processes=2, memory='100G',resource_spec='select=1:ncpus=28:mem=110GB')
print(cluster.job_script())
cluster.scheduler.close()

#!/bin/bash

#!/usr/bin/env bash
#PBS -N dask-worker
#PBS -q mpi_1
#PBS -A equinox
#PBS -l select=1:ncpus=28:mem=100GB
#PBS -l walltime=24:00:00
#PBS -m n
JOB_ID=${PBS_JOBID%.*}



/home1/datahome/aponte/.miniconda3/envs/equinox/bin/python -m distributed.cli.dask_worker tcp://10.148.0.39:45376 --nthreads 4 --nprocs 2 --memory-limit 25.00GB --name dask-worker--${JOB_ID}-- --death-timeout 60 --interface  ib0 

#!/bin/bash

#!/usr/bin/env bash
#PBS -N dask-worker
#PBS -q mpi_1
#PBS -A equinox
#PBS -l select=1:ncpus=28:mem=110GB
#PBS -l walltime=24:00:00
#PBS -m n
JOB_ID=${PBS_JOBID%.*}



/home1/datahome/aponte/.miniconda3/envs/equinox/bin/python -m distributed.cli.dask_worker tcp://10.148.0.39:51114 --nthreads 1 --nprocs 2 --memory-limit 50.00GB --name dask-worker--${JOB_ID}-- --death-timeout 60 --interface  ib0 

#!/bin/bash

#!/usr/bin/env bash
#PBS -N dask-worker
#PBS -q mpi_1
#PBS -A equinox
#PBS -l select=1:ncpus=28:mem=110GB
#PBS -l walltime=24:00:00
#PBS -m n
JOB_ID=${PBS_JOBID%.*

<Future finished result=None>

In [4]:
from dask_jobqueue import PBSCluster